In [18]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import csv
import glob
import cv2
import os

from sklearn.decomposition import PCA
from numpy import linalg as LA
from os.path import join

In [44]:
class point: 
    def __init__(self,x,y,z):
        self.x = x
        self.y = y
        self.z = z
    def position(self):
        return [self.x, self.y, self.z]
    def distance(self,other):
        x_distance = self.x - other.x
        y_distance = self.y - other.y
        z_distance = self.z - other.z
        return LA.norm([x_distance,y_distance,z_distance])
    def projZ(self):
        return self.x, self.y
    def projX(self):
        return self.y, self.z
    def projY(self):
        return self.x, self.z
    def proj_plane(self, vector):
        cord1 = np.dot([self.x,self.y,self.z],vector[:,0])
        cord2 = np.dot([self.x,self.y,self.z],vector[:,1])
        return cord1, cord2

class line: 
    def __init__(self,point_1,point_2):
        self.point_1 = point_1
        self.point_2 = point_2
    def distance(self):
        x_distance = self.point_1.x - self.point_2.x
        y_distance = self.point_1.y - self.point_2.y
        z_distance = self.point_1.z - self.point_2.z
        return LA.norm([x_distance,y_distance,z_distance])

In [45]:
def csv2pointdic(path):
    data = []
    with open(path, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for i, row in enumerate(spamreader):
            if i>4 :
                data.append(row[0].split(','))
    for row in data:
        del row[0]
        del row[-1]
    whole_points = {}
    for time,row in enumerate(data): 
        time_points = {}
        for point_num,position in enumerate(row):
            try:
                x,y,z = position.split('|')
                x,y,z = float(x),float(y),float(z)
                time_points[point_num] = point(x,y,z)
            except Exception as e :
                print(position)
                time_points = None
                break
        whole_points[time] = time_points
    return whole_points


In [46]:
def get_pointnum(whole_points):
    return len(whole_points[0].keys())

In [47]:
def get_framenum(whole_points):
    return len(whole_points.keys())

In [48]:
def scale_checker(whole_points):
    max_x,max_y,max_z = -np.inf,-np.inf,-np.inf
    min_x,min_y,min_z = np.inf,np.inf,np.inf
    for _,time_points in whole_points.items():
        if time_points is None :
            continue
        for _,point in time_points.items():
            if point.x < min_x:
                min_x = point.x
            elif point.x > max_x:
                max_x = point.x
            if point.y < min_y:
                min_y = point.y
            elif point.y > max_y:
                max_y = point.y
            if point.z < min_z:
                min_z = point.z
            elif point.z > max_z:
                max_z = point.z
    return [[min_x-200,max_x+200],[min_y-200,max_y+200],[min_z-200,max_z+200]]

In [49]:
def plot3d_frame(whole_points,frame,scale,path):
    flg = plt.figure()
    ax = flg.gca(projection='3d')
    time_points = whole_points[frame]
    plt.suptitle(f'3d position')
    if time_points is None:
        plt.savefig(path)
    else :
        X,Y,Z = [],[],[]
        for _,point in time_points.items():
            X.append(point.x)
            Y.append(point.y)
            Z.append(point.z)
        ax.scatter(X,Y,Z,c=list(range(0,get_pointnum(whole_points))))
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.set_xlim3d(*scale[0])
        ax.set_ylim3d(*scale[1])
        ax.set_zlim3d(*scale[2])
        plt.savefig(path)
    plt.close()


In [50]:
def plot2d_frame(whole_points,frame,scale,path,plane):
    flg = plt.figure()
    ax = flg.gca()
    time_points = whole_points[frame]
    plt.suptitle(f'2d position in {plane} plane')
    if time_points is None:
        plt.savefig(path)
    else :
        cord1,cord2 = [],[]
        for _,point in time_points.items():
            if plane=='X':
                cord1_num, cord2_num = point.projX()
            elif plane=='Y':
                cord1_num, cord2_num = point.projY()
            elif plane=='Z':
                cord1_num, cord2_num = point.projZ()
            else :
                cord1_num, cord2_num = point.proj_plane(plane)
            cord1.append(cord1_num)
            cord2.append(cord2_num)
        ax.scatter(cord1,cord2,c=list(range(0,get_pointnum(whole_points))))
        ax.set_xlabel('cord1')
        ax.set_ylabel('cord2')
        if plane=='X':
            cord1_index, cord2_index = 1,2
            ax.set_xlim(*scale[cord1_index])
            ax.set_ylim(*scale[cord2_index])
        elif plane=='Y':
            cord1_index, cord2_index = 0,2
            ax.set_xlim(*scale[cord1_index])
            ax.set_ylim(*scale[cord2_index])
        elif plane=='Z':
            cord1_index, cord2_index = 0,1
            ax.set_xlim(*scale[cord1_index])
            ax.set_ylim(*scale[cord2_index])
        else :
            ax.set_xlim(scale[0])
            ax.set_ylim(scale[1])
        plt.savefig(path)
    plt.close()

In [51]:
def plot_save(frame_num,whole_points,path,proj=None):
    plt.ioff()
    os.makedirs(path, exist_ok=True)
    scale=scale_checker(whole_points)
    for i in range(frame_num):
        if proj is None :
            plot3d_frame(whole_points,frame=i,scale=scale,path=join(path,f'{i}.jpg'))
        elif proj in ['X','Y','Z'] :
            plot2d_frame(whole_points,frame=i,scale=scale,path=join(path,f'{i}.jpg'),plane=proj)
        else : #project to plane
            scale_1pc = [np.dot([scale[0][0], scale[1][0], scale[2][0]],proj[:,0]),np.dot([scale[0][1], scale[1][1], scale[2][1]],proj[:,0])]
            scale_2pc = [np.dot([scale[0][0], scale[1][0], scale[2][0]],proj[:,1]),np.dot([scale[0][1], scale[1][1], scale[2][1]],proj[:,1])]
            scale = (scale_1pc,scale_2pc)
            plot2d_frame(whole_points,frame=i,scale=scale,path=join(path,f'{i}.jpg'),plane=proj)


In [52]:
def images2video(image_path,video_path,fps):
    img_array = []
    size = (1,1)
    for filename in glob.glob(join(image_path,'*.jpg')):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    out = cv2.VideoWriter(video_path,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [53]:
whole_points=csv2pointdic('example2.csv')
frame_num = get_framenum(whole_points)

In [56]:
def get_pca(whole_points):
    x = []
    for _,stamp in whole_points.items():
        if stamp is None:
            continue
        for _,point in stamp.items():
            x.append(point.position())
    pca = PCA(n_components=2)
    pca.fit(x)
    pc_vector = pca.components_.T
    pc_ratio = pca.explained_variance_ratio_
    return pc_vector, pc_ratio

[0.53168494 0.40562608]


In [30]:
plot_save(frame_num,whole_points,'ximages',proj='X')
images2video('ximages','xproj_visualize.avi',30)
plot_save(frame_num,whole_points,'yimages',proj='Y')
images2video('yimages','yproj_visualize.avi',30)
plot_save(frame_num,whole_points,'zimages',proj='Z')
images2video('zimages','zporj_visualize.avi',30)
plot_save(frame_num,whole_points,'images')
images2video('images','3d_visualize.avi',30)

<ipython-input-8-3f2ae3ffd28e>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  flg = plt.figure()


In [41]:
def check_videos_open(video_list):
    for cap in video_list:
        if not cap.isOpened():
            return False
    return True

In [42]:
def release_videos(video_list):
    for cap in video_list:
        cap.release()

In [43]:
def horizontal_concat_videos(videos,output):
    cap_list = []
    try :
        for video in videos:
            cap_list.append(cv2.VideoCapture(video))
        frameWidth = int(cap_list[0].get(cv2.CAP_PROP_FRAME_WIDTH))* len(cap_list)
        frameHeight = int(cap_list[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap_list[0].get(cv2.CAP_PROP_FPS))
        size = (frameWidth,frameHeight)
        out = cv2.VideoWriter(output,fourcc =cv2.VideoWriter_fourcc(*'DIVX'), fps = fps, frameSize =size)
        while check_videos_open(cap_list):
            both = 0 #initalize
            for i,cap in enumerate(cap_list):
                ret, frame = cap.read()
                if ret is False:
                    release_videos([*cap_list,out])
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                    return
                if i == 0:
                    both = frame
                else:
                    both = np.concatenate((both, frame), axis=1)
            out.write(both)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        release_videos([*cap_list,out])
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    except:
        release_videos([*cap_list])
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [44]:
def vertical_concat_videos(videos,output):
    cap_list = []
    try :
        for video in videos:
            cap_list.append(cv2.VideoCapture(video))
        frameWidth = int(np.max(list(map(lambda x : x.get(cv2.CAP_PROP_FRAME_WIDTH),cap_list))))
        frameHeight = int(np.sum(list(map(lambda x : x.get(cv2.CAP_PROP_FRAME_HEIGHT),cap_list))))
        fps = int(cap_list[0].get(cv2.CAP_PROP_FPS))
        size = (frameWidth,frameHeight)
        out = cv2.VideoWriter(output,fourcc =cv2.VideoWriter_fourcc(*'DIVX'), fps = fps, frameSize =size)
        while check_videos_open(cap_list):
            both = 0 #initalize
            for i,cap in enumerate(cap_list):
                ret, frame = cap.read()
                if ret is False:
                    release_videos([*cap_list,out])
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                    return
                if i == 0:
                    both = frame
                else:
                    h,w,c= both.shape
                    h1,w1,c1 = frame.shape
                    if w < w1: # resize right img to left size
                        both = cv2.resize(both,(w1,h))
                    elif w > w1:
                        frame = cv2.resize(frame,(w,h))
                    both = np.concatenate((both, frame), axis=0)
            out.write(both)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        release_videos([*cap_list,out])
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    except:
        release_videos([*cap_list])
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [45]:
horizontal_concat_videos(['example_cam1.avi','example_cam2.avi'],'camera.avi')
horizontal_concat_videos(['xproj_visualize.avi','yproj_visualize.avi','zporj_visualize.avi','3d_visualize.avi'],'visualize.avi')
vertical_concat_videos(['camera.avi','visualize.avi'],'output.avi')